***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-f9c0  GCE       5                                       RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
from numpy.linalg import norm
import math

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan 13 18:33 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [7]:
# Get blob names of parguet files to read
bucket_name = 'ir_project_lssl' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if '.parquet' in b.name:
        paths.append(full_path+b.name)

***GCP setup is complete!***

# Building an inverted index (without stemming, plain tokenizer)

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket.

In [8]:
parquetFile = spark.read.parquet(*paths)

Create an id->title dictionary and export to GCS bucket

In [ ]:
doc_title_dict = parquetFile.select("id", "title").rdd.collectAsMap()
# write out the counter as binary file (pickle it)
title_dict = 'title_dict.pkl'
with open(title_dict, 'wb') as f:
  pickle.dump(doc_title_dict, f)
# upload to gs
#index_src = "anchortextindex.pkl"
index_dst = f'gs://{bucket_name}/gcp_title_dict/{title_dict}'
!gsutil cp $title_dict $index_dst

Read all parquet files into RDDs: one for each components

In [14]:
doc_text_pairs = parquetFile.select("text", "id").rdd
doc_title_pairs = parquetFile.select("title", "id").rdd
doc_anchor_text_pairs = parquetFile.select("anchor_text", "id").rdd

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [ ]:
# Count number of wiki pages
parquetFile.count()

Import the Inverted Index class

In [ ]:
%cd -q /home/dataproc
!ls inverted_index_gcp.py

In [ ]:
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [ ]:
from inverted_index_gcp import InvertedIndex

Helper functions to generate Index attributes using MapReduce and Pyspark

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
stemmer = PorterStemmer()

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def doc_len(text, id):
  ''' Count the amount of words in `text` DL that are not included in
  `all_stopwords` and return entries that will go into Doc Len dictionary.
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    Tuple (doc_id, len(doc))
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  filtered =  [stemmer.stem(word) for word in tokens if word not in all_stopwords]
  results = []
  #if len(filtered) != 0:
  results.append((id,len(filtered)))
  return results

def norm_doc(text, id, doc_lens,w2df_dict):
  ''' Calculate the document norm after the text has been tokenized and stop_word were removed.
  Used for Cosine Similarity calculation.
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
    doc_lens: dict
      index.Dl id->len
    w2df_dict: dict
      index.df term->doc_freq
  Returns:
  --------
    List of tuples
      (doc_id, doc_norm)
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  filtered =  [stemmer.stem(word) for word in tokens if word not in all_stopwords]
  freq = defaultdict(int)
  results = []
  for word in filtered:
    freq[word] += 1
  dl = doc_lens[id]
  normlized_tfidf = [(wordfreq /dl) * math.log(len(doc_lens) / w2df_dict[word], 10) for word, wordfreq in freq.items() if word in w2df_dict.keys()]
  norma = norm(normlized_tfidf)
  results.append((id,norma))
  return results

def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  filtered =  [stemmer.stem(word) for word in tokens if word not in all_stopwords]
  freq = defaultdict(int)
  results = []
  for word in filtered:
    freq[word] += 1
  for term in freq.items():
    results.append((term[0],(id,term[1])))
  return results

def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  srtdpl = sorted(unsorted_pl,key=lambda x: x[1])
  return srtdpl


def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  df_rdd = postings.groupByKey().mapValues(lambda x: len(list(x)[0]))
  return df_rdd

def partition_postings_and_write(postings,prefix):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
    prefix: string
     The location of the index in the bucket.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  hashed = postings.map(lambda x: (token2bucket_id(x[0]),(x[0],x[1])))
  buckets = hashed.groupByKey().map(lambda row: InvertedIndex.write_a_posting_list(row, bucket_name,prefix))
  return buckets

TEXT INDEX

Calculate the components using the above functions

In [ ]:
# time the index creation time
t_start = time()
# calculate doc len
doc_lens = doc_text_pairs.flatMap(lambda x: doc_len(x[0], x[1])).collectAsMap()
# word counts map
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
doc_norm = doc_text_pairs.flatMap(lambda x: norm_doc(x[0], x[1],doc_lens,w2df_dict)).collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered,"postings_gcp_text_wnorm").collect()
index_const_time = time() - t_start

Check index_const_time in minutes

In [ ]:
index_const_time/60

Collect posting locations into a dictionary

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp_text_wnorm'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

Save calculated components into the inverted index attributes and export to index to bucket

In [ ]:
# Create inverted index instance
textinverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
textinverted.posting_locs = super_posting_locs
textinverted.DL = doc_lens
textinverted.DN = doc_norm
# Add the token - df dictionary to the inverted index
textinverted.df = w2df_dict
# write the global stats out
textinverted.write_index('.', 'textindex')
# upload to gs
index_src = "textindex.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp_text_wnorm/{index_src}'
!gsutil cp $index_src $index_dst

TITLE INDEX

Calculate the components using the above functions

In [ ]:
# time the index creation time
t_start = time()
# calculate doc len
doc_lens = doc_title_pairs.flatMap(lambda x: doc_len(x[0], x[1])).collectAsMap()
# word counts map
word_counts = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
#postings_filtered = postings.filter(lambda x: len(x[1])>25)
w2df = calculate_df(postings)
w2df_dict = w2df.collectAsMap()
# calculate doc norm
doc_norm = doc_title_pairs.flatMap(lambda x: norm_doc(x[0], x[1],doc_lens,w2df_dict)).collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(postings,"postings_gcp_title_wnorm").collect()
index_const_time = time() - t_start

Check index_const_time in minutes

In [ ]:
index_const_time/60

Collect posting locations into a dictionary

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp_title_wnorm'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

Save calculated components into the inverted index attributes and export to index to bucket

In [ ]:
# Create inverted index instance
titleinverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
titleinverted.posting_locs = super_posting_locs
titleinverted.DL = doc_lens
titleinverted.DN = doc_norm
# Add the token - df dictionary to the inverted index
titleinverted.df = w2df_dict
# write the global stats out
titleinverted.write_index('.', 'titleindex')
# upload to gs
index_src = "titleindex.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp_title_wnorm/{index_src}'
!gsutil cp $index_src $index_dst

Anchor Text Index

Pre processing function of anchor text input

In [ ]:
def pre_process_anchor_text(doc_anchor_text_pairs):
  ''' Pre process anchor_text data by removing ids and concatenating anchor_text string into one string.
  Parameters:
  -----------
    doc_anchor_text_pairs: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and 
      'anchor_text'.
  Returns:
  --------
    doc_anchor_text_pairs: RDD
      An RDD where each row consists of one wikipedia article with 'id' and
      'anchor_text' concatenated as strings.
  '''
  anchor = doc_anchor_text_pairs.map(lambda row:([text[1] for text in row[0]], row[1]))
  anchor_str = anchor.map(lambda row:(' '.join(row[0]), row[1]))
  return anchor_str

Calculate the components using the above functions

In [ ]:
# time the index creation time
t_start = time()
# word counts map
pre_proc_anchor = pre_process_anchor_text(doc_anchor_text_pairs)
doc_lens = pre_proc_anchor.flatMap(lambda x: doc_len(x[0], x[1])).collectAsMap()
word_counts = pre_proc_anchor.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
#postings_filtered = postings.filter(lambda x: len(x[1])>25)
w2df = calculate_df(postings)
w2df_dict = w2df.collectAsMap()
# calculate doc norm
doc_norm = pre_proc_anchor.flatMap(lambda x: norm_doc(x[0], x[1],doc_lens,w2df_dict)).collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(postings,"postings_gcp_anchor_wnorm").collect()
index_const_time = time() - t_start

Check index_const_time in minutes

In [ ]:
index_const_time/60

Collect posting locations into a dictionary

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp_anchor_wnorm'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

Save calculated components into the inverted index attributes and export to index to bucket


In [ ]:
# Create inverted index instance
anchorinverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
anchorinverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
anchorinverted.df = w2df_dict
anchorinverted.DL = doc_lens
anchorinverted.DN = doc_norm
# write the global stats out
anchorinverted.write_index('.', 'anchortextindex')
# upload to gs
index_src = "anchortextindex.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp_anchor_wnorm/{index_src}'
!gsutil cp $index_src $index_dst

# PageRank

In [ ]:
def generate_graph(pages):
  ''' Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and 
      'anchor_text'.
  Returns:
  --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the 
      second entry is the destination page id. No duplicates should be present. 
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph 
      created by the wikipedia links. No duplicates should be present. 
  '''
  grpdcouples = pages.map(lambda x: (x[1],list(set([id[0] for id in x[0]]))))
  edges = grpdcouples.flatMap(lambda k_v: [(k_v[0], x) for x in k_v[1]])
  vertices = edges.flatMap(lambda x: [(x[0],),(x[1],)]).distinct()
  return edges, vertices

In [ ]:
t_start = time()
# construct the graph 
edges, vertices = generate_graph(doc_anchor_text_pairs)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
#pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr')
pr_time = time() - t_start
pr.show()

# Page Views

In [ ]:
# Paths
# Using user page views (as opposed to spiders and automated traffic) for the 
# month of August 2021
pv_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
p = Path(pv_path) 
pv_name = p.name
pv_temp = f'{p.stem}-4dedup.txt'
pv_clean = f'{p.stem}.pkl'
# Download the file (2.3GB) 
!wget -N $pv_path
# Filter for English pages, and keep just two fields: article ID (3) and monthly 
# total number of page views (5). Then, remove lines with article id or page 
# view values that are not a sequence of digits.
!bzcat $pv_name | grep "^en\.wikipedia" | cut -d' ' -f3,5 | grep -P "^\d+\s\d+$" > $pv_temp
# Create a Counter (dictionary) that sums up the pages views for the same 
# article, resulting in a mapping from article id to total page views.
wid2pv = Counter()
with open(pv_temp, 'rt') as f:
  for line in f:
    parts = line.split(' ')
    wid2pv.update({int(parts[0]): int(parts[1])})
# write out the counter as binary file (pickle it)
with open(pv_clean, 'wb') as f:
  pickle.dump(wid2pv, f)
# read in the counter
# with open(pv_clean, 'rb') as f:
#   wid2pv = pickle.loads(f.read())